In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x
def H4(n,m):
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return (S(base6(n,m)^(1/(m-2))).polynomial())
def deltaStar6(n,m):
    jay=j(n,m)
    djay=x*derivative(jay,x)
    numerator=L(djay^(2*m))
    denominator=L(jay^(2*m-2)*(jay-2^6*m^3)^m)
    base=L(numerator/denominator)
    xp=1/(m-2)
    power=S(base^xp)
    return D(power).truncate(n+1)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def deltaDiamond(n,m):
    return D(S(expand((H4(n+2,m)^3)/j(n+2,m)))).truncate(n+1)
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)
def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))/1728
def last(list):return list[-1]

In [3]:
for m in [3..12]:
    print "-----------------------------------------------------------------------"
    print deltaStar6(5,m)

-----------------------------------------------------------------------
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x
-----------------------------------------------------------------------
-13762560*x^5 + 262144*x^4 + 3072*x^3 - 128*x^2 + x
-----------------------------------------------------------------------
-12041250*x^5 + 7688000/3*x^4 + 27900*x^3 - 360*x^2 + x
-----------------------------------------------------------------------
1610612736*x^5 + 8388608*x^4 + 147456*x^3 - 768*x^2 + x
-----------------------------------------------------------------------
22718141950*x^5 - 11524800*x^4 + 543900*x^3 - 1400*x^2 + x
-----------------------------------------------------------------------
195066593280*x^5 - 740294656/3*x^4 + 1585152*x^3 - 2304*x^2 + x
-----------------------------------------------------------------------
1239560731710*x^5 - 1414276416*x^4 + 3921372*x^3 - 3528*x^2 + x
-----------------------------------------------------------------------
18790481920000/3*x^5 - 1677721

In [5]:
poly=deltaStar6(100,3)
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-nieburTau(n)]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
import time
import pickle
wfile = open('/Users/barrybrent/21mar21no4.txt','w') # star series to precision 25. 
data=[]
for m in [3..95]:
    start=time.time()
    ds=deltaStar6(25,m)
    data=data+[[m,ds]]
    finish=time.time()
    print([m,ds.degree(),finish-start])
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 25, 0.15616703033447266]
[4, 25, 0.11806011199951172]
[5, 25, 0.25821805000305176]
[6, 25, 0.1958150863647461]
[7, 25, 0.3166348934173584]
[8, 25, 0.24268102645874023]
[9, 25, 0.3446469306945801]
[10, 25, 0.35289502143859863]
[11, 25, 0.42510199546813965]
[12, 25, 0.36300206184387207]
[13, 25, 0.45627903938293457]
[14, 25, 0.4559600353240967]
[15, 25, 0.6302790641784668]
[16, 25, 0.6030900478363037]
[17, 25, 0.7845029830932617]
[18, 25, 0.6629018783569336]
[19, 25, 0.825659990310669]
[20, 25, 0.7925970554351807]
[21, 25, 0.886387825012207]
[22, 25, 0.7320830821990967]
[23, 25, 0.9627571105957031]
[24, 25, 1.0474529266357422]
[25, 25, 1.1314749717712402]
[26, 25, 1.242163896560669]
[27, 25, 1.2652201652526855]
[28, 25, 1.331974983215332]
[29, 25, 1.4391238689422607]
[30, 25, 1.2307379245758057]
[31, 25, 1.652076005935669]
[32, 25, 1.5306549072265625]
[33, 25, 1.928678035736084]
[34, 25, 1.8131921291351318]
[35, 25, 1.8694908618927002]
[36, 25, 1.9130921363830566]
[37, 25, 2.34418916

In [2]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/21mar21no4.txt','r') # star series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/20mar21no11.txt','w') # star polynomials
for qpower in [1..25]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower,rl]] 
    finish = time.time()
    print([qpower,rl.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[1, 0, 0.3571140766143799]
[2, 3, 0.10448908805847168]
[3, 6, 0.06981301307678223]
[4, 9, 0.06689810752868652]
[5, 12, 0.06833410263061523]
[6, 15, 0.06736898422241211]
[7, 18, 0.0677640438079834]
[8, 21, 0.06684517860412598]
[9, 24, 0.06699705123901367]
[10, 27, 0.06604290008544922]
[11, 30, 0.0671541690826416]
[12, 33, 0.06955599784851074]
[13, 36, 0.06796598434448242]
[14, 39, 0.06800103187561035]
[15, 42, 0.0677490234375]
[16, 45, 0.0683908462524414]
[17, 48, 0.06740903854370117]
[18, 51, 0.06867098808288574]
[19, 54, 0.07049298286437988]
[20, 57, 0.07140207290649414]
[21, 60, 0.0705270767211914]
[22, 63, 0.07137393951416016]
[23, 66, 0.07374191284179688]
[24, 69, 0.0726621150970459]
[25, 72, 0.07155394554138184]
